<a href="https://colab.research.google.com/github/nethranatarajan3/nethranatarajan3.github.io/blob/main/portfolio_code/CC8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with Supermarket Big Data




In [2]:
# Import Required Libraries
import pandas as pd
import numpy as np
import altair as alt

## Loading Data



In [3]:
# Load cleaned prices and items data
prices_df = pd.read_parquet('https://autocpi-public.s3.eu-west-2.amazonaws.com/pp434/pp434_semi_anonymised_prices.parquet')
items_df = pd.read_parquet('https://autocpi-public.s3.eu-west-2.amazonaws.com/pp434/pp434_semi_anonymised_items.parquet')

In [4]:
# Show sample rows from prices_df and items_df
prices_df.sample(5)

,store_id,product_id,date,price,unit_price,loyalty_price,original_price
8849227,2,8124018,2025-03-10,3.75,£10.71 / kg,NaN,3.75
20690192,2,7625336,2024-09-24,3.00,15p / 100ml,2.5,NaN
6826590,1,268821142,2024-05-16,3.25,8.67 per kg,NaN,NaN
18826949,2,7861018,2024-03-06,2.30,92p / 100g,NaN,2.30
21418344,2,8079106,2024-05-12,9.00,£1.80 / 100g,NaN,9.00


In [5]:
items_df.sample(5)

,store_id,product_id,segment_code,description
57515,1,282247340,CP0114601,"YOGHURT, FROM MILK OF COWS"
21893,3,4254459,CP0117903,"CRISPS, POTATO"
10469,4,217559011,CP0211002,"SPIRITS, WHISKY"
15021,1,300400448,CP0112201,"MEAT OF COWS, FRESH, CHILLED OR FROZEN"
46551,4,113550011,CP0122001,COFFEE


</br></br>


# Merging the data



In [6]:
# Merge prices and items on store_id and product_id
df = pd.merge(prices_df, items_df, on=['store_id', 'product_id'], how='inner')
df.sample(5)

,store_id,product_id,date,price,unit_price,loyalty_price,original_price,segment_code,description
3625127,8,963894931449315,2024-01-13,4.55,£2.28/100g,NaN,NaN,CP0122001,COFFEE
1604631,2,8027200,2024-06-25,4.25,50p / 100g,3.25,NaN,CP0114601,"YOGHURT, FROM MILK OF COWS"
2485765,2,8004790,2024-07-17,1.65,£1.57 / 100g,NaN,1.65,CP0118501,"CHOCOLATE, NO ADDITIONAL FILLING (EXCL. ASSORT..."
2842722,3,1000342163062,2024-07-15,1.95,£11.34/kg,NaN,NaN,CP0119101,"PRE-COOKED DISHES BASED ON MEAT, FISH, VEGETAB..."
3255628,5,4088600304632,2025-03-15,0.75,£0.15 / 100g,NaN,0.75,CP0119304,PRE-MADE SAUCES (E.G. BOLOGNESE)


In [7]:
df.description.unique()

array(['RICE, IN ALL FORMS (EXCL. RICE FLOUR)', 'FLOUR, WHEAT-BASED',
       'BREAD, WHITE', 'BREAD, BROWN OR SEEDED',
       'BREAD ROLLS, BUNS, BAGUETTES AND OTHER LOAVES',
       'FLATBREADS, THINS AND PITTAS',
       'BREAD SIDE DISHES (E.G. GARLIC BREAD)',
       'OTHER BREAKFAST BAKERY PRODUCTS', 'BISCUITS, SWEET',
       'BISCUITS, SAVOURY', 'CAKES, TARTS AND SWEET PIES',
       'BREAKFAST CEREALS', 'CEREAL BARS AND CEREAL-BASED SNACKS',
       'OATS AND PORRIDGE', 'PASTA AND NOODLES, DRY OR FRESH',
       'PASTA AND NOODLES, PACKET OR POT', 'COUSCOUS',
       'MEAT OF COWS, FRESH, CHILLED OR FROZEN',
       'MEAT OF PIGS, FRESH, CHILLED OR FROZEN',
       'MEAT OF GOATS, LAMBS AND SHEEP, FRESH, CHILLED OR FROZEN',
       'MEAT OF CHICKEN, FRESH, CHILLED OR FROZEN',
       'COOKED HAM AND CONTINENTAL MEATS (E.G. SALAMI)',
       'COOKED POULTRY, SLICES AND DELI FOODS',
       'PORK, DRIED, SALTED OR SMOKED',
       'SAUSAGES AND SIMILAR MEAT PRODUCTS',
       'BREADED CHICKEN AN

</br></br></br></br>

# Ice Creams Chart

Let's chart all frozen fruit prices over time

In [8]:
ice_cream_df = df.query("description == 'ICE CREAM BARS, LOLLIES AND CONES'")

# This gives us a df with every price observation for frozen fruit products
# But we want the mean price over time

ice_cream_avg_df = ice_cream_df.groupby(['date']).agg({'price': 'mean'}).reset_index()


alt.Chart(ice_cream_avg_df).mark_line(
    interpolate='monotone',
).encode(
    x=alt.X('date:T', title=''),
    y=alt.Y('price:Q', title='Mean price'),
).properties(
    title='Average Price of Ice Cream Bars, Lollies and Cones',
)


alt.Chart(...)

In [16]:
# Selecting 5 essential products to compare
products_to_compare = [
    'BREAD',
    'MILK',
    'EGGS',
    'BUTTER',
    'ICE CREAM'
]

# Filtering for products containing these keywords
multi_product_df = pd.DataFrame()

for product in products_to_compare:
    product_df = df[df['description'].str.contains(product, case=False, na=False)].copy()
    product_df['product_category'] = product.title()  # Add category label
    multi_product_df = pd.concat([multi_product_df, product_df], ignore_index=True)

# Calculating average price by date and product category
price_trends = multi_product_df.groupby(['date', 'product_category'])['price'].mean().reset_index()

# Converting date to datetime
price_trends['date'] = pd.to_datetime(price_trends['date'])

print("Price trends data:")
print(price_trends.head(20))

# Saving to CSV for Vega-Lite
price_trends.to_csv('multi_product_price_trends.csv', index=False)

# Creating the chart with Altair
import altair as alt

chart1 = alt.Chart(price_trends).mark_line(
    strokeWidth=2,
    point=True
).encode(
    x=alt.X('date:T', title='Date'),
    y=alt.Y('price:Q', title='Average Price (£)'),
    color=alt.Color('product_category:N',
                    title='Product Category',
                    scale=alt.Scale(scheme='category10')),
    tooltip=[
        alt.Tooltip('date:T', title='Date', format='%Y-%m-%d'),
        alt.Tooltip('product_category:N', title='Product'),
        alt.Tooltip('price:Q', title='Avg Price', format='.2f')
    ]
).properties(
    title='Price Trends: Essential Products Over Time',
    width=600,
    height=350
)

# Displaying the chart
display(chart1)


# Saving as Vega-Lite JSON
chart1.save('Week8_Chart1.json')


Price trends data:
         date product_category     price
0  2023-07-01            Bread  1.934282
1  2023-07-01           Butter  2.852187
2  2023-07-01             Eggs  2.450000
3  2023-07-01        Ice Cream  3.388983
4  2023-07-01             Milk  2.129755
5  2023-07-02            Bread  2.011592
6  2023-07-02           Butter  3.359231
7  2023-07-02             Eggs  2.467391
8  2023-07-02        Ice Cream  3.599519
9  2023-07-02             Milk  2.160748
10 2023-07-03            Bread  2.024898
11 2023-07-03           Butter  3.320000
12 2023-07-03             Eggs  2.812500
13 2023-07-03        Ice Cream  3.388889
14 2023-07-03             Milk  2.555000
15 2023-07-04            Bread  1.657400
16 2023-07-04             Eggs  2.350000
17 2023-07-04        Ice Cream  3.272727
18 2023-07-04             Milk  2.375000
19 2023-07-05            Bread  2.156020


alt.Chart(...)


✅ Saved Week8_Chart1.json


In [25]:
# Getting top 10 most common product categories
top_categories = df['description'].value_counts().head(10).index.tolist()

print("Top 10 product categories:")
for i, cat in enumerate(top_categories, 1):
    print(f"{i}. {cat}")

# Filtering for these categories
heatmap_df = df[df['description'].isin(top_categories)].copy()

# Calculating average price by store and product category
store_product_prices = heatmap_df.groupby(['store_id', 'description'])['price'].mean().reset_index()

# Creating store labels
store_product_prices['store_label'] = 'Store ' + store_product_prices['store_id'].astype(str)

# Conditional truncation: if > 30 chars, use first word; else use full name (up to 30)
def smart_truncate(desc):
    if len(str(desc)) > 30:
        return str(desc).split()[0]  # First word only
    else:
        return str(desc)[:30]  # Truncate at 30 chars

store_product_prices['description_short'] = store_product_prices['description'].apply(smart_truncate)

print("\nStore-Product price matrix:")
print(store_product_prices[['description', 'description_short']].drop_duplicates())

# Saving to CSV for Vega-Lite
store_product_prices.to_csv('store_product_heatmap.csv', index=False)

# Creating heatmap with Altair
import altair as alt

chart2 = alt.Chart(store_product_prices).mark_rect().encode(
    x=alt.X('store_label:N', title='Store'),
    y=alt.Y('description_short:N', title='Product Category'),
    color=alt.Color('price:Q',
                    title='Avg Price (£)',
                    scale=alt.Scale(scheme='viridis')),
    tooltip=[
        alt.Tooltip('store_label:N', title='Store'),
        alt.Tooltip('description:N', title='Full Product Name'),
        alt.Tooltip('price:Q', title='Avg Price', format='.2f')
    ]
).properties(
    title='Price Comparison Heatmap: Top 10 Products Across Stores',
    width=500,
    height=400
)

# Displaying the chart
display(chart2)

# Saving as Vega-Lite JSON
chart2.save('Week8_Chart2.json')


Top 10 product categories:
1. PRE-COOKED DISHES BASED ON MEAT, FISH, VEGETABLES, PASTA OR OTHER CEREALS, CHILLED OR FROZEN
2. CAKES, TARTS AND SWEET PIES
3. YOGHURT, FROM MILK OF COWS
4. COFFEE
5. CHOCOLATE, NO ADDITIONAL FILLING (EXCL. ASSORTMENT AND COOKING)
6. CONDIMENTS
7. CRISPS, POTATO
8. SPICES AND CULINARY HERBS (EXCL. SEEDS)
9. WINE, RED
10. WINE, WHITE

Store-Product price matrix:
                                         description  \
0                        CAKES, TARTS AND SWEET PIES   
1  CHOCOLATE, NO ADDITIONAL FILLING (EXCL. ASSORT...   
2                                             COFFEE   
3                                         CONDIMENTS   
4                                     CRISPS, POTATO   
5  PRE-COOKED DISHES BASED ON MEAT, FISH, VEGETAB...   
6            SPICES AND CULINARY HERBS (EXCL. SEEDS)   
7                                          WINE, RED   
8                                        WINE, WHITE   
9                         YOGHURT, FROM MILK O

alt.Chart(...)


✅ Saved Week8_Chart2.json
